<a href="https://colab.research.google.com/github/Abhish2702/BTP-Depression-Detection-/blob/main/btp_using_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
d_tweets_dataset=pd.read_csv("/content/clean_d_tweets.csv")
non_d_tweets_dataset=pd.read_csv("/content/clean_non_d_tweets.csv")
d_tweets_dataset.tweet = d_tweets_dataset.tweet.fillna('')
non_d_tweets_dataset.tweet = non_d_tweets_dataset.tweet.fillna('')

In [ ]:
d_tweets=d_tweets_dataset['tweet']

In [ ]:
non_d_tweets=non_d_tweets_dataset['tweet']

In [ ]:
y1=np.ones((3082,1),dtype="int")

In [ ]:
y2=np.zeros((2505,1),dtype="int")

In [ ]:
y=np.concatenate((y1,y2),axis=0)

In [ ]:
res=pd.DataFrame(y)

In [ ]:
txt=pd.concat([d_tweets,non_d_tweets],axis=0)

In [ ]:
txt.shape
txt=txt.reset_index(drop="true")

In [ ]:
df=pd.concat([txt,res],axis=1,join="inner")

In [ ]:
df.columns=["text","result"]


In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenized_text = [word_tokenize(text.lower()) for text in df['text']]

In [ ]:
from gensim.models import FastText

# Train FastText model
fasttext_model = FastText(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4, sg=1)


In [ ]:
# Get the FastText word vectors
word_vectors = fasttext_model.wv


In [ ]:
# Convert text to FastText word embeddings
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text)
X_sequences = tokenizer.texts_to_sequences(tokenized_text)
X_padded = pad_sequences(X_sequences, maxlen=100)

# X_fasttext_padded = pad_sequences(X_fasttext, maxlen=100)
# X_fasttext_padded
# y = df['result'].values


In [ ]:
# Prepare target variable y (same as before)

# Split the data into train and test sets
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [ ]:
X_fasttext = []
for sentence in tokenized_text:
    embedding = []
    for word in sentence:
        if word in word_vectors:
            embedding.append(word_vectors[word])
    X_fasttext.append(embedding)

# Find the maximum sequence length in X_fasttext
max_sequence_length = max(len(embedding) for embedding in X_fasttext)

# Pad sequences to the maximum length
X_fasttext_padded = pad_sequences(X_fasttext, maxlen=100, dtype='float32', padding='post')
X_fasttext_padded.shape
y = df['result'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_fasttext_padded, y, test_size=0.2, random_state=42)

Fasttext with LSTM

In [ ]:

import pandas as pd
from gensim.models import FastText
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your FastText model
sentences = [text.split() for text in df['text']]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)

embedding_dim = 100  # Should match the vector_size in FastText model

model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=maxlen))
model.add(LSTM(64))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 16s 90ms/step - loss: 0.4731 - accuracy: 0.7671 - val_loss: 0.3024 - val_accuracy: 0.8658
Epoch 2/10
140/140 [==============================] - 12s 86ms/step - loss: 0.2019 - accuracy: 0.9255 - val_loss: 0.2852 - val_accuracy: 0.8792
Epoch 3/10
140/140 [==============================] - 12s 86ms/step - loss: 0.1089 - accuracy: 0.9593 - val_loss: 0.3164 - val_accuracy: 0.8775
Epoch 4/10
140/140 [==============================] - 11s 79ms/step - loss: 0.0689 - accuracy: 0.9770 - val_loss: 0.3552 - val_accuracy: 0.8819
Epoch 5/10
140/140 [==============================] - 12s 87ms/step - loss: 0.0459 - accuracy: 0.9834 - val_loss: 0.4700 - val_accuracy: 0.8757
Epoch 6/10
140/140 [==============================] - 12s 86ms/step - loss: 0.0338 - accuracy: 0.9881 - val_loss: 0.5430 - val_accuracy: 0.8712
Epoch 7/10
140/140 [==============================] - 14s 100ms/step - loss: 0.0251 - accuracy: 0.9913 - val_l

Using Glove with LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Load the pre-trained GloVe embeddings
glove_file = '/content/glove.6B.100d.txt'  # Change this to the actual path of your downloaded GloVe file
embedding_dim = 100  # Should match the dimensionality of your pre-trained GloVe vectors

# Create an embedding matrix
embedding_matrix = {}
with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_matrix[word] = coefs

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words and word in embedding_matrix:
        embedding_matrix[i] = embedding_matrix[word]

# Build the BiLSTM model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))



Found 6236 unique tokens.


<ipython-input-56-a5059f61375e>:41: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if i < max_words and word in embedding_matrix:


Epoch 1/10
140/140 [==============================] - 12s 70ms/step - loss: 0.6887 - accuracy: 0.5505 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 2/10
140/140 [==============================] - 7s 53ms/step - loss: 0.6879 - accuracy: 0.5531 - val_loss: 0.6922 - val_accuracy: 0.5456
Epoch 3/10
140/140 [==============================] - 12s 83ms/step - loss: 0.6881 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 4/10
140/140 [==============================] - 10s 72ms/step - loss: 0.6878 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 5/10
140/140 [==============================] - 7s 52ms/step - loss: 0.6881 - accuracy: 0.5531 - val_loss: 0.6891 - val_accuracy: 0.5456
Epoch 6/10
140/140 [==============================] - 9s 66ms/step - loss: 0.6878 - accuracy: 0.5531 - val_loss: 0.6893 - val_accuracy: 0.5456
Epoch 7/10
140/140 [==============================] - 8s 56ms/step - loss: 0.6877 - accuracy: 0.5531 - val_loss: 0.6892 - val_accuracy: 0.5

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Using TF-IDF with LSTM

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
X = df['text']
y = df['result']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
max_length = max(max(len(seq) for seq in X_train_seq), max(len(seq) for seq in X_test_seq))
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(LSTM(128))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
126/126 [==============================] - 19s 132ms/step - loss: 0.6891 - accuracy: 0.5485 - val_loss: 0.6920 - val_accuracy: 0.5280
Epoch 2/10
126/126 [==============================] - 19s 154ms/step - loss: 0.6884 - accuracy: 0.5602 - val_loss: 0.6926 - val_accuracy: 0.5280
Epoch 3/10
126/126 [==============================] - 17s 132ms/step - loss: 0.6188 - accuracy: 0.6658 - val_loss: 0.6617 - val_accuracy: 0.6063
Epoch 4/10
126/126 [==============================] - 16s 128ms/step - loss: 0.6003 - accuracy: 0.6412 - val_loss: 0.6453 - val_accuracy: 0.5772
Epoch 5/10
126/126 [==============================] - 16s 126ms/step - loss: 0.6399 - accuracy: 0.5592 - val_loss: 0.6631 - val_accuracy: 0.5280
Epoch 6/10
126/126 [==============================] - 16s 129ms/step - loss: 0.6361 - accuracy: 0.5472 - val_loss: 0.6605 - val_accuracy: 0.5391
Epoch 7/10
126/126 [==============================] - 16s 124ms/step - loss: 0.6327 - accuracy: 0.5629 - val_loss: 0.6594 - val_ac

In [ ]:

y_pred_probs = model.predict(X_test_pad)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

35/35 [==============================] - 3s 54ms/step
Validation Accuracy: 0.5813953488372093
              precision    recall  f1-score   support

           0   0.520747  0.988189  0.682065       508
           1   0.961039  0.242623  0.387435       610

    accuracy                       0.581395      1118
   macro avg   0.740893  0.615406  0.534750      1118
weighted avg   0.760978  0.581395  0.521310      1118



Using LSTM and word2vec

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
max_words = 10000  # Choose based on your dataset
maxlen = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')
data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
embedding_dim = 100  # Should match the vector_size in Word2Vec model

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(LSTM(64))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

Found 6236 unique tokens.
Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 18s 95ms/step - loss: 0.4469 - accuracy: 0.7834 - val_loss: 0.2897 - val_accuracy: 0.8649
Epoch 2/10
140/140 [==============================] - 13s 90ms/step - loss: 0.1972 - accuracy: 0.9239 - val_loss: 0.2688 - val_accuracy: 0.8855
Epoch 3/10
140/140 [==============================] - 13s 90ms/step - loss: 0.1068 - accuracy: 0.9613 - val_loss: 0.3454 - val_accuracy: 0.8810
Epoch 4/10
140/140 [==============================] - 13s 91ms/step - loss: 0.0725 - accuracy: 0.9747 - val_loss: 0.3964 - val_accuracy: 0.8828
Epoch 5/10
140/140 [==============================] - 12s 89ms/step - loss: 0.0539 - accuracy: 0.9796 - val_loss: 0.4369 - val_accuracy: 0.8873
Epoch 6/10
140/140 [==============================] - 12s 84ms/step - loss: 0.0358 - accuracy: 0.9875 - val_loss: 0.5221 - val_accuracy: 0.8712
Epoch 7/10
140/140 [==============================] - 14s 100ms/step - loss: 0.0246 

Using BiLSTM and word2vec

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
max_words = 10000  # Choose based on your dataset
maxlen = 100

In [ ]:

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

In [ ]:
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

Found 6236 unique tokens.


In [ ]:
data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

In [ ]:
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

Found 6236 unique tokens.


In [ ]:
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
embedding_dim = 100  # Should match the vector_size in Word2Vec model

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

Epoch 1/10
140/140 [==============================] - 25s 141ms/step - loss: 0.4511 - accuracy: 0.7821 - val_loss: 0.2841 - val_accuracy: 0.8712
Epoch 2/10
140/140 [==============================] - 19s 136ms/step - loss: 0.1887 - accuracy: 0.9250 - val_loss: 0.2940 - val_accuracy: 0.8784
Epoch 3/10
140/140 [==============================] - 21s 153ms/step - loss: 0.1161 - accuracy: 0.9568 - val_loss: 0.3411 - val_accuracy: 0.8936
Epoch 4/10
140/140 [==============================] - 22s 155ms/step - loss: 0.0665 - accuracy: 0.9770 - val_loss: 0.3665 - val_accuracy: 0.8918
Epoch 5/10
140/140 [==============================] - 22s 155ms/step - loss: 0.0528 - accuracy: 0.9819 - val_loss: 0.4972 - val_accuracy: 0.8631
Epoch 6/10
140/140 [==============================] - 19s 139ms/step - loss: 0.0406 - accuracy: 0.9850 - val_loss: 0.5104 - val_accuracy: 0.8739
Epoch 7/10
140/140 [==============================] - 20s 140ms/step - loss: 0.0306 - accuracy: 0.9877 - val_loss: 0.5530 - val_ac

Using BiLSTM and fasttext

In [ ]:
import pandas as pd
from gensim.models import FastText
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your FastText model
sentences = [text.split() for text in df['text']]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Build the BiLSTM model
embedding_dim = 100  # Should match the vector_size in FastText model

model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 26s 147ms/step - loss: 0.4437 - accuracy: 0.7856 - val_loss: 0.2871 - val_accuracy: 0.8667
Epoch 2/10
140/140 [==============================] - 21s 148ms/step - loss: 0.1910 - accuracy: 0.9266 - val_loss: 0.2884 - val_accuracy: 0.8828
Epoch 3/10
140/140 [==============================] - 20s 141ms/step - loss: 0.1140 - accuracy: 0.9633 - val_loss: 0.3783 - val_accuracy: 0.8506
Epoch 4/10
140/140 [==============================] - 19s 138ms/step - loss: 0.0849 - accuracy: 0.9707 - val_loss: 0.3553 - val_accuracy: 0.8909
Epoch 5/10
140/140 [==============================] - 24s 170ms/step - loss: 0.0489 - accuracy: 0.9810 - val_loss: 0.4780 - val_accuracy: 0.8801
Epoch 6/10
140/140 [==============================] - 19s 139ms/step - loss: 0.0361 - accuracy: 0.9866 - val_loss: 0.5551 - val_accuracy: 0.8631
Epoch 7/10
140/140 [==============================] - 19s 139ms/step - loss: 0.0287 - accuracy: 0.9890 -

Using BiLSTM and Glove

In [ ]:
!pip install glove-python

In [ ]:
!pip install glove_python

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Load the pre-trained GloVe embeddings
glove_file = '/content/glove.6B.100d.txt'  # Change this to the actual path of your downloaded GloVe file
embedding_dim = 100  # Should match the dimensionality of your pre-trained GloVe vectors

# Create an embedding matrix
embedding_matrix = {}
with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_matrix[word] = coefs

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words and word in embedding_matrix:
        embedding_matrix[i] = embedding_matrix[word]

# Build the BiLSTM model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))



Found 6236 unique tokens.


<ipython-input-47-a3cdc3759390>:41: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if i < max_words and word in embedding_matrix:


Epoch 1/10
140/140 [==============================] - 20s 117ms/step - loss: 0.6886 - accuracy: 0.5522 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 2/10
140/140 [==============================] - 16s 111ms/step - loss: 0.6879 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 3/10
140/140 [==============================] - 15s 111ms/step - loss: 0.6881 - accuracy: 0.5531 - val_loss: 0.6893 - val_accuracy: 0.5456
Epoch 4/10
140/140 [==============================] - 18s 132ms/step - loss: 0.6878 - accuracy: 0.5531 - val_loss: 0.6893 - val_accuracy: 0.5456
Epoch 5/10
140/140 [==============================] - 18s 126ms/step - loss: 0.6876 - accuracy: 0.5531 - val_loss: 0.6895 - val_accuracy: 0.5456
Epoch 6/10
140/140 [==============================] - 16s 113ms/step - loss: 0.6877 - accuracy: 0.5531 - val_loss: 0.6891 - val_accuracy: 0.5456
Epoch 7/10
140/140 [==============================] - 19s 133ms/step - loss: 0.6877 - accuracy: 0.5531 - val_loss: 0.6895 - val_ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

35/35 [==============================] - 2s 49ms/step
Validation Accuracy: 0.5456171735241503
              precision    recall  f1-score   support

           0   0.000000  0.000000  0.000000       508
           1   0.545617  1.000000  0.706019       610

    accuracy                       0.545617      1118
   macro avg   0.272809  0.500000  0.353009      1118
weighted avg   0.297698  0.545617  0.385216      1118



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

35/35 [==============================] - 6s 66ms/step
Validation Accuracy: 0.5456171735241503


Using BiLSTM and tf-idf

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Split the data into training and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# TF-IDF Vectorization
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tfidf_vectorizer = TfidfVectorizer(max_features=max_words)
tfidf_train = tfidf_vectorizer.fit_transform(train_data['text'])
tfidf_val = tfidf_vectorizer.transform(val_data['text'])

# Padding sequences
x_train = pad_sequences(tfidf_train.toarray(), maxlen=maxlen)
x_val = pad_sequences(tfidf_val.toarray(), maxlen=maxlen)

y_train = train_data['result'].values
y_val = val_data['result'].values

# Build the BiLSTM model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))




Epoch 1/10
140/140 [==============================] - 30s 166ms/step - loss: 0.6884 - accuracy: 0.5531 - val_loss: 0.6905 - val_accuracy: 0.5456
Epoch 2/10
140/140 [==============================] - 20s 146ms/step - loss: 0.6885 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 3/10
140/140 [==============================] - 24s 174ms/step - loss: 0.6881 - accuracy: 0.5531 - val_loss: 0.6902 - val_accuracy: 0.5456
Epoch 4/10
140/140 [==============================] - 21s 148ms/step - loss: 0.6881 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 5/10
140/140 [==============================] - 23s 167ms/step - loss: 0.6881 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 6/10
140/140 [==============================] - 20s 146ms/step - loss: 0.6881 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 7/10
140/140 [==============================] - 23s 163ms/step - loss: 0.6877 - accuracy: 0.5531 - val_loss: 0.6892 - val_ac

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

35/35 [==============================] - 2s 37ms/step
Validation Accuracy: 0.5456171735241503
              precision    recall  f1-score   support

           0   0.000000  0.000000  0.000000       508
           1   0.545617  1.000000  0.706019       610

    accuracy                       0.545617      1118
   macro avg   0.272809  0.500000  0.353009      1118
weighted avg   0.297698  0.545617  0.385216      1118



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Using BERT and word2vec

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.2 MB/s eta 0:00:00


In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel, BertConfig
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D, Concatenate

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Tokenize using BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert sentences to BERT input format
bert_input = bert_tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='np')

# Load pre-trained BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Generate BERT embeddings
with torch.no_grad():
    bert_outputs = bert_model(**bert_input)

# Extract embeddings from BERT output
bert_embeddings = bert_outputs['last_hidden_state']

# Average pooling over the sequence dimension
bert_avg_pooling = np.mean(bert_embeddings, axis=1)

# Combine Word2Vec and BERT embeddings
combined_embeddings = np.concatenate([word2vec_model.wv[word] for word in df['text']], axis=0)
combined_embeddings = np.concatenate([combined_embeddings, bert_avg_pooling], axis=1)

# Build the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=combined_embeddings.shape[1]))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(combined_embeddings, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Found 6236 unique tokens.


NameError: ignored

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Concatenate, Embedding, GlobalAveragePooling1D
import torch
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)
# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Tokenize using BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert sentences to BERT input format
bert_input = bert_tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Load pre-trained BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Generate BERT embeddings
with torch.no_grad():
    bert_outputs = bert_model(**bert_input)

# Extract embeddings from BERT output
bert_embeddings = bert_outputs['last_hidden_state']

# Average pooling over the sequence dimension
bert_avg_pooling = np.mean(bert_embeddings.numpy(), axis=1)

# Combine Word2Vec and BERT embeddings
combined_embeddings = np.concatenate([word2vec_model.wv[word] for word in df['text']], axis=0)
combined_embeddings = np.concatenate([combined_embeddings, bert_avg_pooling], axis=1)

# Build the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=combined_embeddings.shape[1]))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(combined_embeddings, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')




KeyError: ignored

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Concatenate, Embedding, GlobalAveragePooling1D
import torch
import os

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Tokenize using BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert sentences to BERT input format
bert_input = bert_tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Check if running on Google Colab and TPU is available
if 'COLAB_TPU_ADDR' in os.environ:
    # Enable TPU acceleration
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
else:
    strategy = None

# Load pre-trained BERT model
with strategy.scope() if strategy else nullcontext():
    bert_model = BertModel.from_pretrained('bert-base-uncased')

    # Generate BERT embeddings
    with torch.no_grad():
        bert_outputs = bert_model(**bert_input)

    # Extract embeddings from BERT output
    bert_embeddings = bert_outputs['last_hidden_state']

# Average pooling over the sequence dimension
bert_avg_pooling = np.mean(bert_embeddings.numpy(), axis=1)

# Combine Word2Vec and BERT embeddings
combined_embeddings = np.concatenate([word2vec_model.wv[word] for word in df['text']], axis=0)
combined_embeddings = np.concatenate([combined_embeddings, bert_avg_pooling], axis=1)

# Build the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=combined_embeddings.shape[1]))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(combined_embeddings, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Found 6236 unique tokens.


NameError: ignored

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Concatenate, Embedding, GlobalAveragePooling1D
import torch
import os
from contextlib import contextmanager

# Define nullcontext for Python versions < 3.7
@contextmanager
def nullcontext(enter_result=None):
    yield enter_result

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Tokenize using BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert sentences to BERT input format
bert_input = bert_tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Check if running on Google Colab and TPU is available
if 'COLAB_TPU_ADDR' in os.environ:
    # Enable TPU acceleration
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
else:
    strategy = nullcontext()

# Load pre-trained BERT model
with strategy:
    bert_model = BertModel.from_pretrained('bert-base-uncased')

    # Generate BERT embeddings
    with torch.no_grad():
        bert_outputs = bert_model(**bert_input)

    # Extract embeddings from BERT output
    bert_embeddings = bert_outputs['last_hidden_state']

# Average pooling over the sequence dimension
bert_avg_pooling = np.mean(bert_embeddings.numpy(), axis=1)

# Combine Word2Vec and BERT embeddings
combined_embeddings = np.concatenate([word2vec_model.wv[word] for word in df['text']], axis=0)
combined_embeddings = np.concatenate([combined_embeddings, bert_avg_pooling], axis=1)

# Build the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=combined_embeddings.shape[1]))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(combined_embeddings, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')




Found 6236 unique tokens.


Using GRU and Word2vec

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D, GRU
import tensorflow as tf

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Use a GRU layer for text representation
embedding_dim = 100  # Adjust based on your Word2Vec model dimension

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(GRU(64))  # You can adjust the number of units based on your needs
# model.add(Dense(64,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model

y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))


Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 17s 100ms/step - loss: 0.4731 - accuracy: 0.7530 - val_loss: 0.2930 - val_accuracy: 0.8703
Epoch 2/10
140/140 [==============================] - 11s 79ms/step - loss: 0.1910 - accuracy: 0.9279 - val_loss: 0.3143 - val_accuracy: 0.8855
Epoch 3/10
140/140 [==============================] - 12s 85ms/step - loss: 0.1157 - accuracy: 0.9582 - val_loss: 0.3097 - val_accuracy: 0.8730
Epoch 4/10
140/140 [==============================] - 12s 84ms/step - loss: 0.0650 - accuracy: 0.9783 - val_loss: 0.4092 - val_accuracy: 0.8837
Epoch 5/10
140/140 [==============================] - 12s 86ms/step - loss: 0.0450 - accuracy: 0.9834 - val_loss: 0.4642 - val_accuracy: 0.8855
Epoch 6/10
140/140 [==============================] - 11s 77ms/step - loss: 0.0374 - accuracy: 0.9875 - val_loss: 0.4783 - val_accuracy: 0.8891
Epoch 7/10
140/140 [==============================] - 11s 80ms/step - loss: 0.0236 - accuracy: 0.9904 - val_l

Using BiGRU and word2vec

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU
import tensorflow as tf

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # Adjust based on your Word2Vec model dimension

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Bidirectional(GRU(64)))  # You can adjust the number of units based on your needs
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 29s 138ms/step - loss: 0.4736 - accuracy: 0.7440 - val_loss: 0.3156 - val_accuracy: 0.8560
Epoch 2/10
140/140 [==============================] - 19s 135ms/step - loss: 0.1965 - accuracy: 0.9237 - val_loss: 0.2827 - val_accuracy: 0.8819
Epoch 3/10
140/140 [==============================] - 21s 147ms/step - loss: 0.1069 - accuracy: 0.9638 - val_loss: 0.3350 - val_accuracy: 0.8739
Epoch 4/10
140/140 [==============================] - 19s 133ms/step - loss: 0.0590 - accuracy: 0.9808 - val_loss: 0.4061 - val_accuracy: 0.8837
Epoch 5/10
140/140 [==============================] - 19s 135ms/step - loss: 0.0364 - accuracy: 0.9881 - val_loss: 0.4216 - val_accuracy: 0.8855
Epoch 6/10
140/140 [==============================] - 21s 152ms/step - loss: 0.0465 - accuracy: 0.9834 - val_loss: 0.4576 - val_accuracy: 0.8766
Epoch 7/10
140/140 [==============================] - 21s 150ms/step - loss: 0.0285 - accuracy: 0.9906 -

Using BiGRU and fasttext

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=212eaad7fd5ed77ee8745b2fd69ec43fa3fa8bd3cec301b0c07d7174be93ecdf
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import fasttext
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU
import tensorflow as tf

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your FastText model
sentences = [text.split() for text in df['text']]
fasttext_model = fasttext.train_unsupervised(sentences, model='skipgram', dim=100, epoch=10)

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # Should match the FastText embedding dimension

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, trainable=False))
model.add(Bidirectional(GRU(64)))  # You can adjust the number of units based on your needs
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with FastText embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = fasttext_model[word]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Found 6236 unique tokens.


TypeError: ignored

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import FastText
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your FastText model
sentences = [text.split() for text in df['text']]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # Should match the FastText embedding dimension

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, trainable=False))
model.add(Bidirectional(GRU(64)))  # You can adjust the number of units based on your needs
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with FastText embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = fasttext_model.wv[word]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 20s 114ms/step - loss: 0.6031 - accuracy: 0.6657 - val_loss: 0.5506 - val_accuracy: 0.7174
Epoch 2/10
140/140 [==============================] - 15s 106ms/step - loss: 0.5120 - accuracy: 0.7480 - val_loss: 0.5065 - val_accuracy: 0.7513
Epoch 3/10
140/140 [==============================] - 15s 108ms/step - loss: 0.4998 - accuracy: 0.7590 - val_loss: 0.4694 - val_accuracy: 0.7907
Epoch 4/10
140/140 [==============================] - 19s 136ms/step - loss: 0.4880 - accuracy: 0.7608 - val_loss: 0.4722 - val_accuracy: 0.7961
Epoch 5/10
140/140 [==============================] - 16s 112ms/step - loss: 0.4734 - accuracy: 0.7713 - val_loss: 0.4619 - val_accuracy: 0.7889
Epoch 6/10
140/140 [==============================] - 16s 115ms/step - loss: 0.4629 - accuracy: 0.7753 - val_loss: 0.4408 - val_accuracy: 0.8086
Epoch 7/10
140/140 [==============================] - 15s 110ms/step - loss: 0.4534 - accuracy: 0.7850 -

Using BiGRU and Glove

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Download GloVe embeddings (you can choose a different dimension, e.g., 50, 100, 200, 300)
glove_file = 'glove.6B.100d.txt'
glove_w2v_file = 'glove.6B.100d.w2v.txt'

# Convert GloVe format to Word2Vec format
glove2word2vec(glove_file, glove_w2v_file)

# Load the GloVe Word2Vec model
glove_model = KeyedVectors.load_word2vec_format(glove_w2v_file, binary=False)

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # Should match the GloVe embedding dimension (in this case, 100)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, trainable=False))
model.add(Bidirectional(GRU(64)))  # You can adjust the number of units based on your needs
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with GloVe embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = glove_model[word]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

Found 6236 unique tokens.


<ipython-input-34-9f34bc848794>:35: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, glove_w2v_file)


Epoch 1/10
140/140 [==============================] - 55s 114ms/step - loss: 0.4779 - accuracy: 0.7659 - val_loss: 0.3924 - val_accuracy: 0.8184
Epoch 2/10
140/140 [==============================] - 15s 110ms/step - loss: 0.3517 - accuracy: 0.8416 - val_loss: 0.3110 - val_accuracy: 0.8640
Epoch 3/10
140/140 [==============================] - 15s 109ms/step - loss: 0.3248 - accuracy: 0.8554 - val_loss: 0.3034 - val_accuracy: 0.8676
Epoch 4/10
140/140 [==============================] - 15s 110ms/step - loss: 0.2977 - accuracy: 0.8747 - val_loss: 0.2917 - val_accuracy: 0.8694
Epoch 5/10
140/140 [==============================] - 15s 110ms/step - loss: 0.2896 - accuracy: 0.8785 - val_loss: 0.3098 - val_accuracy: 0.8587
Epoch 6/10
140/140 [==============================] - 15s 111ms/step - loss: 0.2631 - accuracy: 0.8881 - val_loss: 0.3185 - val_accuracy: 0.8649
Epoch 7/10
140/140 [==============================] - 15s 110ms/step - loss: 0.2503 - accuracy: 0.8924 - val_loss: 0.2769 - val_ac

In [ ]:


model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, trainable=False))
model.add(Bidirectional(GRU(64)))  # You can adjust the number of units based on your needs
model.add(Dense(64,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with GloVe embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = glove_model[word]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Epoch 1/10
140/140 [==============================] - 36s 187ms/step - loss: 0.4663 - accuracy: 0.7722 - val_loss: 0.3473 - val_accuracy: 0.8399
Epoch 2/10
140/140 [==============================] - 18s 128ms/step - loss: 0.3295 - accuracy: 0.8599 - val_loss: 0.3019 - val_accuracy: 0.8640
Epoch 3/10
140/140 [==============================] - 22s 157ms/step - loss: 0.2990 - accuracy: 0.8725 - val_loss: 0.2966 - val_accuracy: 0.8757
Epoch 4/10
140/140 [==============================] - 22s 154ms/step - loss: 0.2848 - accuracy: 0.8789 - val_loss: 0.2807 - val_accuracy: 0.8846
Epoch 5/10
140/140 [==============================] - 29s 207ms/step - loss: 0.2590 - accuracy: 0.8908 - val_loss: 0.3446 - val_accuracy: 0.8587
Epoch 6/10
140/140 [==============================] - 25s 174ms/step - loss: 0.2378 - accuracy: 0.8991 - val_loss: 0.2778 - val_accuracy: 0.8891
Epoch 7/10
140/140 [==============================] - 18s 130ms/step - loss: 0.2188 - accuracy: 0.9103 - val_loss: 0.3033 - val_ac

Using BiGRU and tf-idf

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, GRU

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = TfidfVectorizer(max_features=max_words)
tfidf_matrix = tokenizer.fit_transform(df['text']).toarray()

# Use a Bidirectional GRU layer for text representation
embedding_dim = tfidf_matrix.shape[1]  # Use the number of features from TF-IDF

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=embedding_dim))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(tfidf_matrix, df['result'].values, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Epoch 1/10
140/140 [==============================] - 6s 28ms/step - loss: 0.5767 - accuracy: 0.7389 - val_loss: 0.4321 - val_accuracy: 0.8524
Epoch 2/10
140/140 [==============================] - 3s 22ms/step - loss: 0.3188 - accuracy: 0.9112 - val_loss: 0.3109 - val_accuracy: 0.8649
Epoch 3/10
140/140 [==============================] - 2s 16ms/step - loss: 0.2005 - accuracy: 0.9427 - val_loss: 0.2796 - val_accuracy: 0.8685
Epoch 4/10
140/140 [==============================] - 2s 14ms/step - loss: 0.1423 - accuracy: 0.9604 - val_loss: 0.2699 - val_accuracy: 0.8748
Epoch 5/10
140/140 [==============================] - 2s 18ms/step - loss: 0.1067 - accuracy: 0.9693 - val_loss: 0.2807 - val_accuracy: 0.8685
Epoch 6/10
140/140 [==============================] - 2s 17ms/step - loss: 0.0833 - accuracy: 0.9781 - val_loss: 0.2894 - val_accuracy: 0.8757
Epoch 7/10
140/140 [==============================] - 3s 23ms/step - loss: 0.0670 - accuracy: 0.9828 - val_loss: 0.2911 - val_accuracy: 0.8784

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Dense

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset

tokenizer = TfidfVectorizer(max_features=max_words)
tfidf_matrix = tokenizer.fit_transform(df['text']).toarray()

# Use a Dense neural network for text representation
embedding_dim = tfidf_matrix.shape[1]  # Use the number of features from TF-IDF

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=embedding_dim))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(tfidf_matrix, df['result'].values, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)

# Accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy}')

# Other metrics
print(classification_report(y_val, y_pred,digits=6))

Epoch 1/10
140/140 [==============================] - 3s 17ms/step - loss: 0.5886 - accuracy: 0.7843 - val_loss: 0.4429 - val_accuracy: 0.8551
Epoch 2/10
140/140 [==============================] - 2s 16ms/step - loss: 0.3255 - accuracy: 0.9114 - val_loss: 0.3152 - val_accuracy: 0.8631
Epoch 3/10
140/140 [==============================] - 2s 12ms/step - loss: 0.2033 - accuracy: 0.9432 - val_loss: 0.2804 - val_accuracy: 0.8640
Epoch 4/10
140/140 [==============================] - 2s 11ms/step - loss: 0.1431 - accuracy: 0.9602 - val_loss: 0.2700 - val_accuracy: 0.8676
Epoch 5/10
140/140 [==============================] - 1s 7ms/step - loss: 0.1070 - accuracy: 0.9689 - val_loss: 0.2703 - val_accuracy: 0.8757
Epoch 6/10
140/140 [==============================] - 1s 7ms/step - loss: 0.0839 - accuracy: 0.9763 - val_loss: 0.2884 - val_accuracy: 0.8694
Epoch 7/10
140/140 [==============================] - 1s 7ms/step - loss: 0.0673 - accuracy: 0.9828 - val_loss: 0.2989 - val_accuracy: 0.8739
Ep

Using GRU with fasttext

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import FastText
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your FastText model
sentences = [text.split() for text in df['text']]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # Should match the FastText embedding dimension

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, trainable=False))
model.add(GRU(64))  # You can adjust the number of units based on your needs
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with FastText embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = fasttext_model.wv[word]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 14s 65ms/step - loss: 0.5743 - accuracy: 0.6861 - val_loss: 0.5051 - val_accuracy: 0.7549
Epoch 2/10
140/140 [==============================] - 9s 68ms/step - loss: 0.5053 - accuracy: 0.7552 - val_loss: 0.4808 - val_accuracy: 0.7791
Epoch 3/10
140/140 [==============================] - 8s 55ms/step - loss: 0.4870 - accuracy: 0.7621 - val_loss: 0.4703 - val_accuracy: 0.7907
Epoch 4/10
140/140 [==============================] - 9s 66ms/step - loss: 0.4835 - accuracy: 0.7648 - val_loss: 0.4671 - val_accuracy: 0.7952
Epoch 5/10
140/140 [==============================] - 9s 61ms/step - loss: 0.4740 - accuracy: 0.7686 - val_loss: 0.4806 - val_accuracy: 0.7710
Epoch 6/10
140/140 [==============================] - 8s 59ms/step - loss: 0.4699 - accuracy: 0.7749 - val_loss: 0.4533 - val_accuracy: 0.7979
Epoch 7/10
140/140 [==============================] - 9s 67ms/step - loss: 0.4652 - accuracy: 0.7729 - val_loss: 0.

Using GRU with Glove

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Load the pre-trained GloVe embeddings
glove_file = '/content/glove.6B.100d.txt'  # Change this to the actual path of your downloaded GloVe file
embedding_dim = 100  # Should match the dimensionality of your pre-trained GloVe vectors

# Create an embedding matrix
embedding_matrix = {}
with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_matrix[word] = coefs

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words and word in embedding_matrix:
        embedding_matrix[i] = embedding_matrix[word]


model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(GRU(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))



Found 6236 unique tokens.


<ipython-input-72-1a4869993b54>:41: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if i < max_words and word in embedding_matrix:


Epoch 1/10
140/140 [==============================] - 12s 71ms/step - loss: 0.6884 - accuracy: 0.5522 - val_loss: 0.6898 - val_accuracy: 0.5456
Epoch 2/10
140/140 [==============================] - 8s 58ms/step - loss: 0.6878 - accuracy: 0.5531 - val_loss: 0.6893 - val_accuracy: 0.5456
Epoch 3/10
140/140 [==============================] - 9s 62ms/step - loss: 0.6881 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 4/10
140/140 [==============================] - 9s 67ms/step - loss: 0.6883 - accuracy: 0.5531 - val_loss: 0.6898 - val_accuracy: 0.5456
Epoch 5/10
140/140 [==============================] - 7s 53ms/step - loss: 0.6877 - accuracy: 0.5531 - val_loss: 0.6891 - val_accuracy: 0.5456
Epoch 6/10
140/140 [==============================] - 9s 66ms/step - loss: 0.6879 - accuracy: 0.5531 - val_loss: 0.6891 - val_accuracy: 0.5456
Epoch 7/10
140/140 [==============================] - 7s 53ms/step - loss: 0.6877 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.545

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Using GRU with tf-idf

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text using TF-IDF
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

vectorizer = TfidfVectorizer(max_features=max_words, stop_words=ENGLISH_STOP_WORDS)
tfidf_matrix = vectorizer.fit_transform(df['text']).toarray()

# Tokenize the text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # You can adjust the dimension based on your needs

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, trainable=False))
model.add(GRU(64, return_sequences=True))  # You can adjust the number of units based on your needs
model.add(GRU(32))  # You can adjust the number of units based on your needs
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with TF-IDF embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = tfidf_matrix[:, word_index[word] - 1]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_pred,digits=6))

InvalidParameterError: ignored

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text using TF-IDF
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

vectorizer = TfidfVectorizer(max_features=max_words, stop_words=ENGLISH_STOP_WORDS)
tfidf_matrix = vectorizer.fit_transform(df['text']).toarray()

# Tokenize the text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # You can adjust the dimension based on your needs

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen))
model.add(Bidirectional(GRU(64, return_sequences=True)))
model.add(Bidirectional(GRU(32)))
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with TF-IDF weights
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = tfidf_matrix[:, word_index[word] - 1]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


InvalidParameterError: ignored

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text using TF-IDF
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

vectorizer = TfidfVectorizer(max_features=max_words, stop_words=ENGLISH_STOP_WORDS)
tfidf_matrix = vectorizer.fit_transform(df['text']).toarray()

# Tokenize the text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Use a Bidirectional GRU layer for text representation
embedding_dim = tfidf_matrix.shape[1]  # Adjust based on the TF-IDF matrix dimension

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, weights=[tfidf_matrix], trainable=False))
model.add(Bidirectional(GRU(64, return_sequences=True)))
model.add(Bidirectional(GRU(32)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


InvalidParameterError: ignored

Using SVM and tf-idf

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(df['text'], df['result'], test_size=0.2, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
x_train_tfidf = vectorizer.fit_transform(x_train)
x_val_tfidf = vectorizer.transform(x_val)

from sklearn.linear_model import RidgeClassifier
ridge_model = RidgeClassifier()
ridge_model.fit(x_train_tfidf, y_train)
y_pred_ridge = ridge_model.predict(x_val_tfidf)

# Evaluate the model
accuracy_ridge = accuracy_score(y_val, y_pred_ridge)
print(f'Validation Accuracy (Ridge Classifier): {accuracy_ridge}')
print("Classification Report (Ridge Classifier):")
print(classification_report(y_val, y_pred_ridge,digits=6))


Validation Accuracy (Ridge Classifier): 0.8416815742397138
Classification Report (Ridge Classifier):
              precision    recall  f1-score   support

           0   0.868597  0.767717  0.815047       508
           1   0.823617  0.903279  0.861611       610

    accuracy                       0.841682      1118
   macro avg   0.846107  0.835498  0.838329      1118
weighted avg   0.844055  0.841682  0.840453      1118



In [ ]:
from sklearn.metrics import classification_report

# ...

# Evaluate the model
print("Classification Report (SVM):")
print(classification_report(y_val, y_pred,digits=6))


Classification Report (SVM):
              precision    recall  f1-score   support

           0   0.845992  0.789370  0.816701       508
           1   0.833851  0.880328  0.856459       610

    accuracy                       0.838998      1118
   macro avg   0.839921  0.834849  0.836580      1118
weighted avg   0.839367  0.838998  0.838394      1118



Using Random Forest and tf-idf

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train_tfidf, y_train)

# Predictions on validation set
y_pred_rf = rf_model.predict(x_val_tfidf)

# Evaluate the model
accuracy_rf = accuracy_score(y_val, y_pred_rf)
print(f'Validation Accuracy (Random Forest): {accuracy_rf}')


Validation Accuracy (Random Forest): 0.8112701252236136


In [ ]:
print("Classification Report (Random Forest):")
print(classification_report(y_val, y_pred_rf))

Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.88      0.68      0.76       508
           1       0.77      0.92      0.84       610

    accuracy                           0.81      1118
   macro avg       0.83      0.80      0.80      1118
weighted avg       0.82      0.81      0.81      1118



Using Naive Bayes and tf-idf

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Train Multinomial Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(x_train_tfidf, y_train)

# Predictions on validation set
y_pred_nb = nb_model.predict(x_val_tfidf)

# Evaluate the model
accuracy_nb = accuracy_score(y_val, y_pred_nb)
print(f'Validation Accuracy (Multinomial Naive Bayes): {accuracy_nb}')


Validation Accuracy (Multinomial Naive Bayes): 0.8300536672629696


In [ ]:
# ...

# Evaluate the model
print("Classification Report (Multinomial Naive Bayes):")
print(classification_report(y_val, y_pred_nb))


Classification Report (Multinomial Naive Bayes):
              precision    recall  f1-score   support

           0       0.91      0.70      0.79       508
           1       0.79      0.94      0.86       610

    accuracy                           0.83      1118
   macro avg       0.85      0.82      0.82      1118
weighted avg       0.84      0.83      0.83      1118



Using Logistic Regression and tf-idf

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train_tfidf, y_train)

# Predictions on validation set
y_pred_lr = lr_model.predict(x_val_tfidf)

# Evaluate the model
accuracy_lr = accuracy_score(y_val, y_pred_lr)
print(f'Validation Accuracy (Logistic Regression): {accuracy_lr}')


Validation Accuracy (Logistic Regression): 0.8416815742397138


In [ ]:
print("Classification Report (Logistic Regression):")
print(classification_report(y_val, y_pred_lr,digits=6))

Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           0   0.842650  0.801181  0.821393       508
           1   0.840945  0.875410  0.857831       610

    accuracy                       0.841682      1118
   macro avg   0.841797  0.838295  0.839612      1118
weighted avg   0.841720  0.841682  0.841274      1118



Using ML and word2vec


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text using Word2Vec
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Generate Word2Vec embeddings for each text
embeddings = np.array([np.mean([word2vec_model.wv[word] for word in text.split() if word in word2vec_model.wv] or [np.zeros(word2vec_model.vector_size)], axis=0) for text in df['text']])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

# ...

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Predictions on validation set
y_pred_rf = rf_model.predict(x_val)

# Evaluate the model
print("Classification Report (Random Forest):")
print(classification_report(y_val, y_pred_rf,digits=6))



Classification Report (Random Forest):
              precision    recall  f1-score   support

           0   0.754491  0.744094  0.749257       508
           1   0.789303  0.798361  0.793806       610

    accuracy                       0.773703      1118
   macro avg   0.771897  0.771228  0.771531      1118
weighted avg   0.773485  0.773703  0.773564      1118



In [ ]:
from sklearn.naive_bayes import MultinomialNB

# ...

# Train Multinomial Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)

# Predictions on validation set
y_pred_nb = nb_model.predict(x_val)

# Evaluate the model
print("Classification Report (Multinomial Naive Bayes):")
print(classification_report(y_val, y_pred_nb))

ValueError: ignored

In [ ]:
from sklearn.linear_model import LogisticRegression

# ...

# Train Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train, y_train)

# Predictions on validation set
y_pred_lr = lr_model.predict(x_val)

# Evaluate the model
print("Classification Report (Logistic Regression):")
print(classification_report(y_val, y_pred_lr,digits=6))

Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           0   0.685345  0.625984  0.654321       508
           1   0.709480  0.760656  0.734177       610

    accuracy                       0.699463      1118
   macro avg   0.697412  0.693320  0.694249      1118
weighted avg   0.698513  0.699463  0.697892      1118



In [ ]:
from sklearn.linear_model import RidgeClassifier
ridge_model = RidgeClassifier()
ridge_model.fit(x_train, y_train)
y_pred_ridge = ridge_model.predict(x_val)

# Evaluate the model
accuracy_ridge = accuracy_score(y_val, y_pred_ridge)
print(f'Validation Accuracy (Ridge Classifier): {accuracy_ridge}')
print("Classification Report (Ridge Classifier):")
print(classification_report(y_val, y_pred_ridge,digits=6))

Validation Accuracy (Ridge Classifier): 0.725402504472272
Classification Report (Ridge Classifier):
              precision    recall  f1-score   support

           0   0.708075  0.673228  0.690212       508
           1   0.738583  0.768852  0.753414       610

    accuracy                       0.725403      1118
   macro avg   0.723329  0.721040  0.721813      1118
weighted avg   0.724720  0.725403  0.724696      1118



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Convert text to Word2Vec embeddings
embedding_dim = 100  # Should match the Word2Vec vector_size
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = word2vec_model.wv[word]
        except KeyError:
            pass

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Use different classifiers

## SVM



Found 6236 unique tokens.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)
y_pred_rf = rf_model.predict(x_val)
accuracy_rf = accuracy_score(y_val, y_pred_rf)
print(f'Validation Accuracy (Random Forest): {accuracy_rf}')
print("Classification Report (Random Forest):")
print(classification_report(y_val, y_pred_rf,digits=6))


Validation Accuracy (Random Forest): 0.8050089445438283
Classification Report (Random Forest):
              precision    recall  f1-score   support

           0   0.826577  0.722441  0.771008       508
           1   0.790801  0.873770  0.830218       610

    accuracy                       0.805009      1118
   macro avg   0.808689  0.798106  0.800613      1118
weighted avg   0.807057  0.805009  0.803314      1118



In [ ]:
# Multinomial Naive Bayes (Note: This classifier is typically used with TF-IDF, not Word2Vec)
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)
y_pred_nb = nb_model.predict(x_val)
accuracy_nb = accuracy_score(y_val, y_pred_nb)
print(f'Validation Accuracy (Multinomial Naive Bayes): {accuracy_nb}')
print("Classification Report (Multinomial Naive Bayes):")
print(classification_report(y_val, y_pred_nb,digits=6))

Validation Accuracy (Multinomial Naive Bayes): 0.5840787119856887
Classification Report (Multinomial Naive Bayes):
              precision    recall  f1-score   support

           0   0.570957  0.340551  0.426634       508
           1   0.588957  0.786885  0.673684       610

    accuracy                       0.584079      1118
   macro avg   0.579957  0.563718  0.550159      1118
weighted avg   0.580778  0.584079  0.561429      1118



In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train, y_train)
y_pred_lr = lr_model.predict(x_val)
accuracy_lr = accuracy_score(y_val, y_pred_lr)
print(f'Validation Accuracy (Logistic Regression): {accuracy_lr}')
print("Classification Report (Logistic Regression):")
print(classification_report(y_val, y_pred_lr,digits=6))

Validation Accuracy (Logistic Regression): 0.7567084078711985
Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           0   0.822404  0.592520  0.688787       508
           1   0.724734  0.893443  0.800294       610

    accuracy                       0.756708      1118
   macro avg   0.773569  0.742981  0.744540      1118
weighted avg   0.769114  0.756708  0.749627      1118



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
Using ridge classifier

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Convert text to Word2Vec embeddings
embedding_dim = 100  # Should match the Word2Vec vector_size
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = word2vec_model.wv[word]
        except KeyError:
            pass

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Use Ridge Classifier
ridge_model = RidgeClassifier()
ridge_model.fit(x_train, y_train)
y_pred_ridge = ridge_model.predict(x_val)

# Evaluate the model
accuracy_ridge = accuracy_score(y_val, y_pred_ridge)
print(f'Validation Accuracy (Ridge Classifier): {accuracy_ridge}')
print("Classification Report (Ridge Classifier):")
print(classification_report(y_val, y_pred_ridge,digits=6))


Found 6236 unique tokens.
Validation Accuracy (Ridge Classifier): 0.7558139534883721
Classification Report (Ridge Classifier):
              precision    recall  f1-score   support

           0   0.885246  0.531496  0.664207       508
           1   0.707257  0.942623  0.808152       610

    accuracy                       0.755814      1118
   macro avg   0.796251  0.737060  0.736179      1118
weighted avg   0.788132  0.755814  0.742746      1118



In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199770 sha256=7bebffa9a1b9334514f2c469df690e6e0aa8559926c8e2c7f81800c2fcb017d8
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


Using fasttext and ML

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import fasttext
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Format data for FastText training
fasttext_data = []
for text, label in zip(df['text'], df['result']):
    fasttext_data.append(f"__label__{label} {text}")

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(fasttext_data, labels, test_size=0.2, random_state=42)

# Save training data to a file
train_file_path = 'fasttext_train.txt'
with open(train_file_path, 'w', encoding='utf-8') as f:
    for item in x_train:
        f.write("%s\n" % item)

# Train FastText model
model = fasttext.train_supervised(input=train_file_path, epoch=10, wordNgrams=2, minCount=1, lr=0.1, dim=100)

# Convert text to FastText embeddings
embedding_dim = 100  # Should match the FastText dimension used in training
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = model.get_word_vector(word)
        except KeyError:
            pass

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Use different classifiers

## RandomForest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)
y_pred_rf = rf_model.predict(x_val)
accuracy_rf = accuracy_score(y_val, y_pred_rf)
print(f'Validation Accuracy (Random Forest): {accuracy_rf}')
print("Classification Report (Random Forest):")
print(classification_report(y_val, y_pred_rf))



## Ridge Classifier
ridge_model = RidgeClassifier()
ridge_model.fit(x_train, y_train)
y_pred_ridge = ridge_model.predict(x_val)
accuracy_ridge = accuracy_score(y_val, y_pred_ridge)
print(f'Validation Accuracy (Ridge Classifier): {accuracy_ridge}')
print("Classification Report (Ridge Classifier):")
print(classification_report(y_val, y_pred_ridge))

## Multinomial Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)
y_pred_nb = nb_model.predict(x_val)
accuracy_nb = accuracy_score(y_val, y_pred_nb)
print(f'Validation Accuracy (Multinomial Naive Bayes): {accuracy_nb}')
print("Classification Report (Multinomial Naive Bayes):")
print(classification_report(y_val, y_pred_nb))

## Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train, y_train)
y_pred_lr = lr_model.predict(x_val)
accuracy_lr = accuracy_score(y_val, y_pred_lr)
print(f'Validation Accuracy (Logistic Regression): {accuracy_lr}')
print("Classification Report (Logistic Regression):")
print(classification_report(y_val, y_pred_lr))


Found 6236 unique tokens.
Validation Accuracy (Random Forest): 0.8050089445438283
Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.83      0.72      0.77       508
           1       0.79      0.87      0.83       610

    accuracy                           0.81      1118
   macro avg       0.81      0.80      0.80      1118
weighted avg       0.81      0.81      0.80      1118



Using glove and ML

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Load pre-trained GloVe embeddings
glove_file_path = '/content/glove.6B.100d.txt'  # Replace with the path to your GloVe file
embedding_dim = 100  # Should match the GloVe embedding dimension

embeddings_index = {}
with open(glove_file_path, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Convert text to GloVe embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Use different classifiers

## RandomForest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)
y_pred_rf = rf_model.predict(x_val)
accuracy_rf = accuracy_score(y_val, y_pred_rf)
print(f'Validation Accuracy (Random Forest): {accuracy_rf}')
print("Classification Report (Random Forest):")
print(classification_report(y_val, y_pred_rf,digits=6))



## Ridge Classifier
ridge_model = RidgeClassifier()
ridge_model.fit(x_train, y_train)
y_pred_ridge = ridge_model.predict(x_val)
accuracy_ridge = accuracy_score(y_val, y_pred_ridge)
print(f'Validation Accuracy (Ridge Classifier): {accuracy_ridge}')
print("Classification Report (Ridge Classifier):")
print(classification_report(y_val, y_pred_ridge,digits=6))

## Multinomial Naive Bayes (Note: This classifier is typically used with TF-IDF, not GloVe)
nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)
y_pred_nb = nb_model.predict(x_val)
accuracy_nb = accuracy_score(y_val, y_pred_nb)
print(f'Validation Accuracy (Multinomial Naive Bayes): {accuracy_nb}')
print("Classification Report (Multinomial Naive Bayes):")
print(classification_report(y_val, y_pred_nb,digits=6))

## Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train, y_train)
y_pred_lr = lr_model.predict(x_val)
accuracy_lr = accuracy_score(y_val, y_pred_lr)
print(f'Validation Accuracy (Logistic Regression): {accuracy_lr}')
print("Classification Report (Logistic Regression):")
print(classification_report(y_val, y_pred_lr,digits=6))


Found 6236 unique tokens.
Validation Accuracy (Random Forest): 0.8050089445438283
Classification Report (Random Forest):
              precision    recall  f1-score   support

           0   0.826577  0.722441  0.771008       508
           1   0.790801  0.873770  0.830218       610

    accuracy                       0.805009      1118
   macro avg   0.808689  0.798106  0.800613      1118
weighted avg   0.807057  0.805009  0.803314      1118

Validation Accuracy (Ridge Classifier): 0.7558139534883721
Classification Report (Ridge Classifier):
              precision    recall  f1-score   support

           0   0.885246  0.531496  0.664207       508
           1   0.707257  0.942623  0.808152       610

    accuracy                       0.755814      1118
   macro avg   0.796251  0.737060  0.736179      1118
weighted avg   0.788132  0.755814  0.742746      1118

Validation Accuracy (Multinomial Naive Bayes): 0.5840787119856887
Classification Report (Multinomial Naive Bayes):
         

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Using Fasttext and ML

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import FastText
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Train FastText model
sentences = [text.split() for text in df['text']]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Prepare feature matrix
X = np.array([np.mean([fasttext_model.wv[word] for word in text.split() if word in fasttext_model.wv] or [np.zeros(fasttext_model.vector_size)], axis=0) for text in df['text']])
y = df['result'].values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Machine Learning Models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),

    'Logistic Regression': LogisticRegression(max_iter=1000)
}

# Train and evaluate models
for name, model in models.items():
    # Use a pipeline if normalization is required
    # model = Pipeline([('scaler', StandardScaler()), ('classifier', model)])

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    # Classification report
    print(f"Classification Report ({name}):")
    print(classification_report(y_val, y_pred,digits=6))

    # Accuracy
    accuracy = accuracy_score(y_val, y_pred)
    print(f'Validation Accuracy ({name}): {accuracy}')


Classification Report (Random Forest):
              precision    recall  f1-score   support

           0   0.698444  0.706693  0.702544       508
           1   0.753311  0.745902  0.749588       610

    accuracy                       0.728086      1118
   macro avg   0.725877  0.726297  0.726066      1118
weighted avg   0.728380  0.728086  0.728212      1118

Validation Accuracy (Random Forest): 0.7280858676207513
Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           0   0.662953  0.468504  0.549020       508
           1   0.644269  0.801639  0.714390       610

    accuracy                       0.650268      1118
   macro avg   0.653611  0.635072  0.631705      1118
weighted avg   0.652758  0.650268  0.639249      1118

Validation Accuracy (Logistic Regression): 0.6502683363148479


In [ ]:
from sklearn.linear_model import RidgeClassifier
ridge_model = RidgeClassifier()
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_val)

# Evaluate the model
accuracy_ridge = accuracy_score(y_val, y_pred_ridge)
print(f'Validation Accuracy (Ridge Classifier): {accuracy_ridge}')
print("Classification Report (Ridge Classifier):")
print(classification_report(y_val, y_pred_ridge,digits=6))

Validation Accuracy (Ridge Classifier): 0.6762075134168157
Classification Report (Ridge Classifier):
              precision    recall  f1-score   support

           0   0.672986  0.559055  0.610753       508
           1   0.678161  0.773770  0.722818       610

    accuracy                       0.676208      1118
   macro avg   0.675573  0.666413  0.666785      1118
weighted avg   0.675809  0.676208  0.671897      1118



Using Glove and ML

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Download GloVe embeddings (choose the appropriate dimension, e.g., 50, 100, 200, 300)
glove_file = 'glove.6B.100d.txt'
glove_w2v_file = 'glove.6B.100d.w2v.txt'

# Convert GloVe format to Word2Vec format
glove2word2vec(glove_file, glove_w2v_file)

# Load the GloVe Word2Vec model
glove_model = KeyedVectors.load_word2vec_format(glove_w2v_file, binary=False)

# Create TF-IDF matrix
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['text']).toarray()

# Generate document embeddings using GloVe
embeddings = []
for text in df['text']:
    words = text.split()
    vectors = [glove_model[word] for word in words if word in glove_model]
    if vectors:
        embeddings.append(np.mean(vectors, axis=0))
    else:
        embeddings.append(np.zeros(glove_model.vector_size))

data = np.array(embeddings)
labels = df['result'].values

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)
from sklearn.ensemble import RandomForestClassifier

# ...



<ipython-input-24-f43037e9a112>:18: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, glove_w2v_file)


Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.87      0.77      0.82       508
           1       0.82      0.90      0.86       610

    accuracy                           0.84      1118
   macro avg       0.85      0.84      0.84      1118
weighted avg       0.84      0.84      0.84      1118



In [ ]:
# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Predictions on validation set
y_pred_rf = rf_model.predict(x_val)

# Evaluate the model
print("Classification Report (Random Forest):")
print(classification_report(y_val, y_pred_rf,digits=6))

Classification Report (Random Forest):
              precision    recall  f1-score   support

           0   0.866962  0.769685  0.815433       508
           1   0.824588  0.901639  0.861394       610

    accuracy                       0.841682      1118
   macro avg   0.845775  0.835662  0.838413      1118
weighted avg   0.843842  0.841682  0.840510      1118



In [ ]:
ridge_model = RidgeClassifier()
ridge_model.fit(x_train, y_train)
y_pred_ridge = ridge_model.predict(x_val)

# Evaluate the model
accuracy_ridge = accuracy_score(y_val, y_pred_ridge)
print(f'Validation Accuracy (Ridge Classifier): {accuracy_ridge}')
print("Classification Report (Ridge Classifier):")
print(classification_report(y_val, y_pred_ridge,digits=6))

Validation Accuracy (Ridge Classifier): 0.8461538461538461
Classification Report (Ridge Classifier):
              precision    recall  f1-score   support

           0   0.862069  0.787402  0.823045       508
           1   0.834862  0.895082  0.863924       610

    accuracy                       0.846154      1118
   macro avg   0.848466  0.841242  0.843485      1118
weighted avg   0.847225  0.846154  0.845349      1118



In [ ]:
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train, y_train)
y_pred_lr = lr_model.predict(x_val)
accuracy_lr = accuracy_score(y_val, y_pred_lr)
print(f'Validation Accuracy (Logistic Regression): {accuracy_lr}')
print("Classification Report (Logistic Regression):")
print(classification_report(y_val, y_pred_lr,digits=6))

Validation Accuracy (Logistic Regression): 0.8497316636851521
Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           0   0.858650  0.801181  0.828921       508
           1   0.843168  0.890164  0.866029       610

    accuracy                       0.849732      1118
   macro avg   0.850909  0.845673  0.847475      1118
weighted avg   0.850202  0.849732  0.849167      1118

